
# Import required libraries

In [8]:
import math
import torch
import json
import numpy as np
import torch.utils.data
import torch.optim as optim
import torch.nn as nn
import random

from auxiliary import load_data, prepare_data_loaders
from algorithms import MLP

# Prepare data, model parameters and dataloaders

In [9]:
json_file = "data.json"
inp_rss_vals, gt_locations = load_data(json_file)  # load_data fonksiyonunu kullanarak veriyi yükle

k=3
number_of_training_iters = 401
batch_size = 32
train_test_split = 0.8

train_loader, test_loader, tensor_x_train, tensor_y_train, tensor_x_test, tensor_y_test = prepare_data_loaders(
    inp_rss_vals, gt_locations, batch_size, train_test_split)

# Step 1: Select k nearest neighbors for each test point

In [10]:
selected_train_indices = []
selected_train_inputs = []
selected_train_outputs = []

for test_point in tensor_x_test:
    distances = torch.norm(tensor_x_train - test_point, dim=1)
    nearest_indices = torch.topk(distances, k=k, largest=False).indices  # Find k nearest neighbors
    selected_train_indices.append(nearest_indices)

    # Collecting the corresponding RSS and locations of nearest neighbors
    selected_train_inputs.extend(tensor_x_train[nearest_indices])
    selected_train_outputs.extend(tensor_y_train[nearest_indices])

selected_train_inputs = torch.stack(selected_train_inputs)
selected_train_outputs = torch.stack(selected_train_outputs)

# Step 2: Train an MLP on the selected training set

In [12]:
selected_train_dataset = torch.utils.data.TensorDataset(selected_train_inputs, selected_train_outputs)
selected_train_loader = torch.utils.data.DataLoader(selected_train_dataset, batch_size=batch_size, shuffle=True)

model = MLP()
model.train()

criterion = nn.MSELoss(reduction='mean')
optimizer = optim.Adam(model.parameters(), lr=1e-3)

for i in range(number_of_training_iters):
    running_loss = 0.0
    for inputs, labels in selected_train_loader:
        optimizer.zero_grad()       # Zero the gradients
        outputs = model(inputs)     # Forward pass
        loss = criterion(outputs, labels)  # Compute loss
        loss.backward()             # Backward pass
        optimizer.step()            # Update weights
        running_loss += loss.item() # Accumulate loss

    if i % 20 == 0:
        print(f'Epoch [{i + 1}/{number_of_training_iters}] running accumulative loss across all batches: {running_loss:.3f}')

Epoch [1/401] running accumulative loss across all batches: 223.772
Epoch [21/401] running accumulative loss across all batches: 63.014
Epoch [41/401] running accumulative loss across all batches: 53.340
Epoch [61/401] running accumulative loss across all batches: 51.873
Epoch [81/401] running accumulative loss across all batches: 51.159
Epoch [101/401] running accumulative loss across all batches: 49.714
Epoch [121/401] running accumulative loss across all batches: 49.089
Epoch [141/401] running accumulative loss across all batches: 48.326
Epoch [161/401] running accumulative loss across all batches: 48.316
Epoch [181/401] running accumulative loss across all batches: 47.864
Epoch [201/401] running accumulative loss across all batches: 47.693
Epoch [221/401] running accumulative loss across all batches: 47.128
Epoch [241/401] running accumulative loss across all batches: 47.429
Epoch [261/401] running accumulative loss across all batches: 49.171
Epoch [281/401] running accumulative lo

# Evaluate the model on the test set

In [13]:
model.eval()
predicted_locations_test = model(tensor_x_test)
#print("Predicted locations using combined Fingerprint and MLP:", predicted_locations_test)

# Save the model parameters

In [14]:
torch.save(model.state_dict(), 'model_parameters.pth')